In [108]:
import torch
import numpy as np
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
from torchvision.transforms import transforms

from imagedataset import ImageDataset

In [109]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [110]:
num_epochs = 2
batch_size = 100
learning_rate = 0.001

In [111]:
norm_transform = transforms.Normalize(
    (132.3501, 127.2977, 131.0638),
    (55.5031, 62.3274, 64.1869)
)

In [112]:
dataset = ImageDataset()
train_size = int(0.75*len(dataset))
test_size  = len(dataset) - train_size
train, test = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test, batch_size=batch_size, shuffle=False)

In [113]:
class ConvNet(nn.Module):
    
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)   # 6 channels out, kernel size 7
        self.pool = nn.MaxPool2d(2, 2)    # 2x2 maxpool (strife). Image now has shape 49x49
        self.conv2 = nn.Conv2d(6, 16, 5)  # 6 channel input, 16 channel output. Image now has dimension 47x47
        self.fc1 = nn.Linear(16*47*47, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 29)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.flatten(start_dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [114]:
model = ConvNet().to(device)
criterion = nn.CrossEntropyLoss() # Includes softmax
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [115]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.float().to(device)
        labels = labels.to(device)

        # Forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward:
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}")

Epoch [1/4], Step [2000/21750], Loss: 2.1295
Epoch [1/4], Step [4000/21750], Loss: 0.5539
Epoch [1/4], Step [6000/21750], Loss: 0.4286
Epoch [1/4], Step [8000/21750], Loss: 0.3627
Epoch [1/4], Step [10000/21750], Loss: 0.0650
Epoch [1/4], Step [12000/21750], Loss: 0.0011
Epoch [1/4], Step [14000/21750], Loss: 0.1373
Epoch [1/4], Step [16000/21750], Loss: 0.2428
